In [ ]:
# USER INPUTS
watershed_files_directory = r"WF_Watershed_Files"
Watershed_Name = "West Fork" # For Chart Title

# Assuming the GHNCD Data Prep script has completed.  The shapefile folder should have a CSV_Precipitation folder with the GHNCD data
# The DSS files should be placed in a subfolder name defined below (Assumed to be in same folder as the watershed shapefile):
DSS_Folder = "DSS-6"

# set variable for GHNCD Search Buffer in Miles 
GHNCD_Search_Buffer_Miles = 10
# Adjust this depending on the size of the watershed and available data

# Geoshift for your final plot (in miles)
plotextentsbuffer_miles = -3
geoshift_miles = 0  # (x axis only)
increase_x_axis = 0  # (x axis only)
shift_x_axis = -5  # (x axis only)
shift_y_axis = -5  # (y axis only)
# Adjust to get labels to fit and create report-quality plots
#color_ramp_vmax = 10  # (inches) #optional, if not set, will be determined by the maximum value in the DSS file

# This is the EPSG code of the CRS of the DSS files (Need to override as PyDSSTools are buggy with CRS)
Project_EPSG = "5070"  # SHOULD BE, EPSG 5070, 5071, or ESRI:102039 as these are all SHG

# List of DSS files and their corresponding event periods
# dss_filename = DSS containing the precipitation data
# event_intervals = List of lists containing the start and end of the event period (Format: 'DDMMMYYYY:HHMM')
# precip_vmax = Maximum precipitation value for the color ramp (inches)
# Send ChatGPT a screenshot of your files and your modeled event periods, and it can edit these quickly for large lists

dss_data_list = [
  
 {'dss_filename': 'AORC_Harvey_LWIRegion4.dss',  'event_intervals': [['10AUG2017:0000', '15SEP2017:0000']], 'precip_vmax': '26'},
 {'dss_filename': 'MRMS_Harvey_LWIRegion4.dss',  'event_intervals': [['10AUG2017:0000', '15SEP2017:0000']], 'precip_vmax': '26'},  

 {'dss_filename': 'AORC_March2016_LWIRegion4.dss',  'event_intervals': [['21FEB2016:0000', '26MAR2016:0000']], 'precip_vmax': '12'},
 {'dss_filename': 'MRMS_March2016_LWIRegion4.dss',  'event_intervals': [['21FEB2016:0000', '26MAR2016:0000']], 'precip_vmax': '12'},

 ]

''' Example of dss_data_list

{'dss_filename': 'AORC_Harvey_LWIRegion4.dss',  'event_intervals': [['10AUG2017:0000', '15SEP2017:0000']], 'precip_vmax': '26'},

 '''

In [ ]:
# Import required Libraries
import subprocess
import sys
import platform

def install_module(module_name):
    try:
        __import__(module_name)
        #print(f"{module_name} is already installed.")
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

def python_version_check(version):
    current_version = ".".join(map(str, sys.version_info[:3]))
    if current_version != version:
        print(f"Current Python version ({current_version}) does not match required version ({version}).")
        print("Please create an environment with Python version: ", version)

# Check Python version # THIS SHOULD CHECK THAT PYTHON VERSION IS 3.9.1 OR GREATER - PLEASE FIX
python_version_check("3.9.1")


def download_and_install_wheel(wheel_url):
    import requests 
    wheel_file = wheel_url.split("/")[-1]
    response = requests.get(wheel_url)
    
    if response.status_code == 200:
        with open(wheel_file, 'wb') as f:
            f.write(response.content)
        subprocess.check_call([sys.executable, "-m", "pip", "install", wheel_file])
        print(f"Successfully installed GDAL from {wheel_file}.")
    else:
        print(f"Failed to download {wheel_file}. HTTP Status Code: {response.status_code}")



def install_osgeo_and_pydss():
    # Check and install osgeo
    try:
        from osgeo import ogr
        print("Successfully imported ogr from osgeo.")
    except ImportError:
        print("Failed to import ogr from osgeo. Attempting to download and install GDAL wheel...")

        # Get Python version and system architecture
        python_version = f"cp{sys.version_info.major}{sys.version_info.minor}"
        arch = 'win_amd64' if platform.architecture()[0] == '64bit' else 'win32'

        # Generate the wheel URL dynamically
        wheel_url = f"https://download.lfd.uci.edu/pythonlibs/archived/GDAL-3.4.3-{python_version}-{python_version}-{arch}.whl"

        download_and_install_wheel(wheel_url)

        # Re-try importing the ogr module
        try:
            from osgeo import ogr
            print("Successfully imported ogr from osgeo.")
        except ImportError:
            print("Still unable to import ogr from osgeo after attempting to install. Please check your environment.")

    # Check and install PyDSS
    try:
        from pydsstools.heclib.dss import HecDss
    except ImportError:
        install_module("pydsstools")
        print("PyDSS not found. If this fails repeatedly, please follow the steps below for all dependencies:")
        print("1. Install essential dependencies: NumPy, pandas, affine, bokeh, chardet")
        print("2. Install C++ Build Tools for Visual Studio 2022 from:")
        print("   https://download.visualstudio.microsoft.com/download/pr/33081bfc-10f1-42d4-8f5a-df6709b8b105/5ae40ab45b17219f8a9c505f5106bbe818bceba5f8e92bd1df5d6db8fa44d820/vs_BuildTools.exe")
        print("3. Download and install the PyDSS Wheel from:")
        print("   https://raw.githubusercontent.com/gyanz/pydsstools/master/dist/pydsstools-2.2-cp39-cp39-win_amd64.whl")


# List of modules to check and install if necessary
modules = ['affine',
 'rasterio',
 'tables',
 'pandas',
 'shutil',
 'numba',
 'numpy',
 'cython',
 'requests',
 'bs4',
 'chardet',
 'concurrent.futures',
 'csv',
 'rioxarray',
 'xarray',
 'pyproj',
 'platform',
 'dask',
 'os',
 'matplotlib',
 'shapely']

for module in modules:
    install_module(module)

# To execute the function:
install_osgeo_and_pydss()

from pydsstools.heclib.dss import HecDss
from pydsstools.heclib.dss.HecDss import Open
from pydsstools.heclib.utils import BoundingBox
import rasterio
import pandas as pd
import numpy as np
import sys
from numba import jit
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import rioxarray as rxr
import geopandas as gpd
import xarray as xr
import os
import shutil
from IPython.display import clear_output
from shapely.geometry import Point
import pyproj
from rasterio.features import geometry_mask
from rasterio.crs import CRS
import io
import shutil
import re
from IPython.display import clear_output  # Importing the clear_output function


In [ ]:
# Logic to loop over each event and produce the output files



# Find .shp file within watershed_files_directory and set as Watershed_Shapefile
import os
for file in os.listdir(watershed_files_directory):
    if file.endswith(".shp"):
        Watershed_Shapefile = os.path.join(watershed_files_directory, file)
        print(Watershed_Shapefile)
    else:
        pass


# Define Paths and Output Files


failed_files = []  # List to hold the filenames of failed files

output_files_folder = os.path.join(os.path.dirname(Watershed_Shapefile), "Output_Files")
# Create output_files_folder if it doesn't exist
if not os.path.exists(output_files_folder):
    os.mkdir(output_files_folder)
    


# Initialize statistics_df before the loop
csv_path = os.path.join(output_files_folder, "Precipitation_Statistics_All_Events.csv")
if not os.path.exists(csv_path):
    statistics_df = pd.DataFrame(columns=['Event_Name', 'Gridded_Data_Label', 'Event_Start_Date', 'Event_End_Date', 'Event_Duration', 'Average_Precipitation', 'Maximum_Precipitation', 'Minimum_Precipitation'])
    statistics_df.to_csv(csv_path, index=False)
else:
    statistics_df = pd.read_csv(csv_path)



# Loop through each DSS file and its event periods
for dss_entry in dss_data_list:
    # Capture stdout
    log_capture_string = io.StringIO()
    sys.stdout = log_capture_string
    DSS_File = r"{}".format(dss_entry["dss_filename"])
    Event_Periods = dss_entry["event_intervals"]
    color_ramp_vmax = dss_entry["precip_vmax"]
    Event_Name = DSS_File.split('_')[1]
    Gridded_Data_Label = DSS_File.split('_')[0]
    
    # Gridded_Data_Label = "MRMS" or "AORC"

    #clear_output(wait=True)
    # Watershed Shapefile Folder + Output_Files + Event Name
    
    event_output_folder = os.path.join(output_files_folder, f"{Event_Name}_{Gridded_Data_Label}_{GHNCD_Search_Buffer_Miles}mi")
    log_file_path = os.path.join(event_output_folder, 'event_log.txt')

    print(f"Working on {DSS_File} for the period {Event_Periods[0][0]} to {Event_Periods[0][1]}")


    # Step 1: Display DSS Catalog and Convert to Pandas Dataframe
    pathname_pattern ="/*/*/*/*/*/*/"

    DSS_Path = os.path.join(os.path.dirname(Watershed_Shapefile), DSS_Folder, DSS_File)

    

    # Check to see if output_files_folder folder exists
    if not os.path.exists(output_files_folder):
        os.mkdir(output_files_folder)

    # If folder event_output_folder exists, delete it and create a new one, otherwise create it
    if os.path.exists(event_output_folder):
        shutil.rmtree(event_output_folder)
        os.mkdir(event_output_folder)
    else:
        os.mkdir(event_output_folder)     



    # Precip_CSV_Path = directory of watershed_shapefile + shapefile + CSV_Precipitation + Precipitation.csv
    Precip_CSV_Path = os.path.join(os.path.dirname(Watershed_Shapefile), "CSV_Precipitation", "Precipitation.csv")
    Total_precipitation_tiff_path = os.path.join(event_output_folder, "Total_precipitation.tif")
    Precip_stations_shapefile_path = os.path.join(event_output_folder, "Precip_stations.shp")
    Precip_stations_csv_path = os.path.join(event_output_folder, "Precip_stations.csv")
    Report_Figure_Path = os.path.join(event_output_folder, f"Report_Figure_{Event_Name}_{Gridded_Data_Label}_{GHNCD_Search_Buffer_Miles}mi.png")
    '''
    PLEASE NOTE 
    in matplotlib, rasters must be masked to the plot extents or they will be distorted

    '''



    # Open DSS File and organize data

    with Open(DSS_Path) as fid:
        path_list = fid.getPathnameList(pathname_pattern,sort=1)
        df = pd.DataFrame(path_list, columns=['Path'])
        # Full pathname catalog as pandas dataframe
        #display(df)

    # Use pandas dataframe to store pathname catalog

    # pathname is based on / separators, use the "/" as delimiter and break pathname into individual components
    pathname_catalog_df = pd.DataFrame(df['Path'].str.split('/').tolist(), columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
    print("Pathname Catalog Dataframe (pathname_catalog_df):")
    #display(pathname_catalog_df)

    # Data is hourly, first 2 pathnames are:
    #/SHG/LWIREGION4/PRECIPITATION/01APR2016:0000/01APR2016:0100/AORC/
    #/SHG/LWIREGION4/PRECIPITATION/01APR2016:0100/01APR2016:0200/AORC/

    '''Sample of pathname_catalog_df dataframe:

    A	B	C	D	E	F	G	H
    0	SHG	LWIREGION4	PRECIPITATION	01APR2016:0000	01APR2016:0100	AORC
    ...
    1584		SHG	LWIREGION4	PRECIPITATION	31MAR2016:2300	31MAR2016:2400	AORC	
    '''

    # Transform the 'E' and 'F' columns to datetime format for proper sorting
    def convert_to_datetime(date_str):
        if ":2400" in date_str:
            date = pd.to_datetime(date_str.replace(":2400", ":0000"), format='%d%b%Y:%H%M')
            date += pd.Timedelta(days=1)
            return date
        return pd.to_datetime(date_str, format='%d%b%Y:%H%M')

    pathname_catalog_df['E-T'] = pathname_catalog_df['E'].apply(convert_to_datetime)
    pathname_catalog_df['F-T'] = pathname_catalog_df['F'].apply(convert_to_datetime)

    # Sort pathname_catalog_df by 'E-T' column (transformed start date)
    pathname_catalog_df = pathname_catalog_df.sort_values(by=['E-T'])

    #Display the sorted pathname_catalog_df
    # display(pathname_catalog_df)  #  Uncomment to debug

    # Start Date = First Pathname's 'E-T' Column
    dss_start_date = pathname_catalog_df['E-T'].iloc[0]
    print(f"DSS Start Date: {dss_start_date}")

    # Timestep = Second Pathname's 'F-T' Column - First Pathname's 'E-T' Column
    dss_timestep = pathname_catalog_df['F-T'].iloc[0] - pathname_catalog_df['E-T'].iloc[0]
    print(f"DSS Timestep: {dss_timestep}")

    # End Date = Last Pathname's 'F-T' Column
    dss_end_date = pathname_catalog_df['F-T'].iloc[-1]
    print(f"DSS End Date: {dss_end_date}")
    print("")

    # Duration = Last Pathname's 'F-T' Column - First Pathname's 'E-T' Column
    dss_duration = pathname_catalog_df['F-T'].iloc[-1] - pathname_catalog_df['E-T'].iloc[0]
    print(f"DSS Total Duration: {dss_duration}")

    # Convert Event_Periods to datetime format
    Event_Periods_dt = [(convert_to_datetime(start), convert_to_datetime(end)) for start, end in Event_Periods]
    print(Event_Periods_dt)

    # Convert the first tuple to a list
    temp_list = list(Event_Periods_dt[0])

    # Modify the list elements
    if temp_list[0] < dss_start_date:
        temp_list[0] = dss_start_date
        print(f"DSS Start Date is later than Event Start Date. Changing Event Start Date to {dss_start_date}")
    if temp_list[1] > dss_end_date:
        temp_list[1] = dss_end_date
        print(f"DSS End Date is earlier than Event End Date. Changing Event End Date to {dss_end_date}")

    # Convert the list back to a tuple and replace the original tuple
    Event_Periods_dt[0] = tuple(temp_list)

    
    # Using Event Periods, filter dataframe to only include pathnames that fall within the event periods
    # Create a new dataframe pathname_catalog_for_event_periods_df
    pathname_catalog_for_event_periods_df = pathname_catalog_df[pathname_catalog_df['E-T'].between(Event_Periods_dt[0][0], Event_Periods_dt[0][1])]

    # Then, exclude E-T and F-T columns
    pathname_catalog_for_event_periods_df = pathname_catalog_for_event_periods_df.drop(columns=['E-T', 'F-T'])

    # Then create a new column with the full recreated pathname
    pathname_catalog_for_event_periods_df['Path'] = pathname_catalog_for_event_periods_df.apply(lambda row: '/'.join(row.values.astype(str)), axis=1)

    # display(pathname_catalog_for_event_periods_df)  # Debug only

    # -------- Previous code block is to create pathname_catalog_for_event_periods_df dataframe --------

    # EVENT-LEVEL TOTAL PRECIPITATION (Export GeoTIFF and Spatial Average Precipitation)


    Event_Name_Title = re.sub(r'(\D)(\d)', r'\1 \2', Event_Name)



    dss_start_title = str(dss_start_date).split(" ")[0].split("-")[1:]
    dss_end_title = str(dss_end_date).split(" ")[0].split("-")[1:]

    # Joining the split parts if needed (for example if you have MM-DD format)
    dss_start_title = '-'.join(dss_start_title)
    dss_end_title = '-'.join(dss_end_title)


    Report_Figure_Title = f"{Watershed_Name}\nGHNCD Stations vs {Gridded_Data_Label} Gridded Precipitation\n {Event_Name_Title}: {dss_start_title} to {dss_end_title}"

        
    # Initialize a variable for total precipitation
    total_precip = None

    tmp_tiff_folder = os.path.join(event_output_folder, "/tmp_tiff")  

    # Loop over each pathname in the filtered dataframe
    count = 0  # Initialize counter
    for pathname in pathname_catalog_for_event_periods_df['Path']:
        # Open the DSS file
        with Open(DSS_Path) as fid:
            # Increment the count
            count += 1

            # Display the current pathname every 10 iterations
            if count % 10 == 0:
                print("Reading pathname:", pathname)
            
            


            # Read the grid data from the DSS file
            ds = fid.read_grid(pathname)
            grid_array = ds.read()
            ds._crs = rasterio.crs.CRS.from_epsg(Project_EPSG)
            profile = ds.profile
            #override profile to project epsg
            profile['crs'] = rasterio.crs.CRS.from_epsg(Project_EPSG)
            rio_dataset = ds.raster._as_rasterio_dataset()
            xarr_data = rxr.open_rasterio(rio_dataset)
            rio_dataset.close()

            # If the total precipitation grid is not initialized, set it to the current grid
            if total_precip is None:
                total_precip = xarr_data
            else:
                # Add the current grid data to the total precipitation grid
                total_precip += xarr_data

            max_precip = total_precip.max().item()

            if count % 10 == 0:
                print("Max Precip:", max_precip)
    
            
    # Count number of pathnames in the filtered dataframe
    print("Number of Pathnames: ", len(pathname_catalog_for_event_periods_df))

    # Convert total_precip to inches
    total_precip_inches = total_precip * 0.0393701

    # Print max precip in MM and inches
    max_precip = total_precip.max().item()
    print("Max Precip: ", max_precip, " MM")
    max_precip_inches = total_precip_inches.max().item()
    print("Max Precip: ", max_precip_inches, " Inches")

    # Write the total precipitation grid to a GeoTIFF file, only if it doesn't already exist
    if not os.path.exists(Total_precipitation_tiff_path):
        total_precip_inches.rio.to_raster(Total_precipitation_tiff_path)
    else:
        print("Total_Precipitation.tif already exists.")

    # THIS IS GOOD, THE TOTAL PRECIP TIFF LOOKS FINE IN QGIS

    # Load the raster file using rioxarray
    total_precip_inches_tiff = rxr.open_rasterio(Total_precipitation_tiff_path)

    # Print max of total_precip_inches_tiff
    print("Total Precipitation GeoTIFF Maximum Value: ", total_precip_inches_tiff.max().item())

    # Display raster crs (SHOULD BE SHG, EPSG: 5070)
    print("Total Precipitation GeoTIFF Coordinate System: ", total_precip_inches_tiff.rio.crs)

    # Plot the total precipitation tiff file (full extents)
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.imshow(total_precip_inches_tiff[0], cmap='GnBu')
    plt.colorbar(cax, ax=ax, label="Inches Precipitation")
    ax.set_title(f"Total Precipitation for {Event_Name}, {Gridded_Data_Label} (Full DSS Extents)")
    f"{Event_Name}_{Gridded_Data_Label}_{GHNCD_Search_Buffer_Miles}mi"
    plt.show()

    # -------- Previous code block is to create total_precip_inches_tiff --------

    # Mask the total precipitation tiff file, calculate statistics and plot sampled data for visual inspection

    # Import watershed shapefile as watershed_SHG (convert to SHG projection, EPSG: 5070)
    watershed_SHG = gpd.read_file(Watershed_Shapefile)
    # Print existing watershed projection
    print("Watershed Original Projection: ", watershed_SHG.crs)
    watershed_SHG = watershed_SHG.to_crs("ESRI:102039")


    # Find the largest polygon by area
    largest_polygon = watershed_SHG.loc[watershed_SHG['geometry'].area.idxmax()]

    # Extract XY coordinates from the largest polygon's exterior
    coords = np.array(largest_polygon.geometry.exterior.coords)

    # Calculate mean, min, and max for X and Y coordinates
    mean_x, mean_y = np.mean(coords[:, 0]), np.mean(coords[:, 1])
    print("Mean X: ", mean_x)
    min_x, min_y = np.min(coords[:, 0]), np.min(coords[:, 1])
    print("Min X: ", min_x)
    max_x, max_y = np.max(coords[:, 0]), np.max(coords[:, 1])
    print("Max X: ", max_x)

    # Set the extents for the plot with a 10% buffer
    plot_extents = [min_x - (max_x - min_x) * 1, max_x + (max_x - min_x) * 1, min_y - (max_y - min_y) * 1, max_y + (max_y - min_y) * 1]


    # Mask the total precipitation tiff file to the plot extents
    # This will ensure that the gridded data scales properly with the watershed shapefile

    from shapely.geometry import box
    import geopandas as gpd

    # Create bounding box geometry from plot extents [minx, maxx, miny, maxy]
    bounding_box = box(plot_extents[0], plot_extents[2], plot_extents[1], plot_extents[3])

    # Create a GeoDataFrame with the bounding box
    bbox_gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=total_precip_inches_tiff.rio.crs)

    # Clip the raster with the bounding box
    clipped_total_precip_inches_tiff = total_precip_inches_tiff.rio.clip(bbox_gdf.geometry)



    # Plot the total precipitation tiff file and watershed boundary in SHG
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.imshow(clipped_total_precip_inches_tiff[0], cmap='GnBu', extent=plot_extents)
    watershed_SHG.plot(ax=ax, facecolor='none', edgecolor='red')
    plt.colorbar(cax, ax=ax, label="Inches Precipitation")
    ax.set_title(f"Total Precipitation for {Event_Name}, {Gridded_Data_Label}, SHG")
    plt.show()
    fig.savefig(os.path.join(event_output_folder, "Report_Figure_TotalPrecip_and_Boundary_Zoomed.png"))

    # -------- Previous code block is to create clipped_total_precip_inches_tiff --------

    # Rasterize the watershed polygon and calculate statistics within the watershed for the event period

    # Import watershed shapefile as watershed_SHG (convert to SHG projection, EPSG: 5070)
    watershed_SHG = gpd.read_file(Watershed_Shapefile)
    # Print existing watershed projection
    print("Watershed Original Projection: ", watershed_SHG.crs)
    watershed_SHG = watershed_SHG.to_crs("ESRI:102039")
    print("Watershed New Projection: ", watershed_SHG.crs)

    # Find the largest polygon by area
    shapes = [(largest_polygon.geometry, 1)]

    # Get extents of watershed_SHG before rasterizing
    # Extract XY coordinates from the largest polygon's exterior
    watershed_shg_coords_before_rasterizing = np.array(largest_polygon.geometry.exterior.coords)

    # calculate extent of coordinates
    watershed_shg_coords_before_rasterizing_extents = [np.min(watershed_shg_coords_before_rasterizing[:, 0]), np.min(watershed_shg_coords_before_rasterizing[:, 1]), np.max(watershed_shg_coords_before_rasterizing[:, 0]), np.max(watershed_shg_coords_before_rasterizing[:, 1])]
    print("Watershed SHG Extents Before Rasterizing: ", watershed_shg_coords_before_rasterizing_extents)



    import rasterio
    from rasterio.mask import mask

    # Read GeoTIFF using rasterio
    with rasterio.open(Total_precipitation_tiff_path) as src:
        # Print CRS of src
        print("Total Precipitation GeoTIFF Coordinate System: ", src.crs)
        geoms = [largest_polygon.geometry.__geo_interface__]
        out_image, out_transform = mask(src, geoms, invert=False, crop=True)


    masked_precip = xr.DataArray(out_image[0], coords={'y': range(out_image.shape[1]), 'x': range(out_image.shape[2])}, dims=['y', 'x'])

    # Now 'masked_precip' contains the same data as 'total_precip_inches_tiff' but masked outside the largest_polygon.geometry


    # Print number of points within the watershed
    print("Number of points within the watershed: ", masked_precip.count().item())

    # 3. Calculate the average, maximum, and minimum precipitation within the watershed for the event period
    avg_precip_watershed = masked_precip.mean().item()
    max_precip_watershed = masked_precip.max().item()
    min_precip_watershed = masked_precip.min().item()

    print(f"Average precipitation within the watershed for the event period: {avg_precip_watershed:.2f} inches")
    print(f"Maximum precipitation within the watershed for the event period: {max_precip_watershed:.2f} inches")
    print(f"Minimum precipitation within the watershed for the event period: {min_precip_watershed:.2f} inches")

    # Create a text file in the event_output_folder to store the statistics
    with open(os.path.join(event_output_folder, "Precipitation_Statistics.txt"), 'w') as f:
        f.write(f"Average precipitation within the watershed for the event period: {avg_precip_watershed:.2f} inches\n")
        f.write(f"Maximum precipitation within the watershed for the event period: {max_precip_watershed:.2f} inches\n")
        f.write(f"Minimum precipitation within the watershed for the event period: {min_precip_watershed:.2f} inches\n")
    
    # Create dataframe of statistics, each loop will append to this dataframe

    # Specify the order of columns
    columns = ['Event_Name', 'Gridded_Data_Label', 'Event_Start_Date', 'Event_End_Date', 'Event_Duration', 'Average_Precipitation', 'Maximum_Precipitation', 'Minimum_Precipitation']
    
    # Populate new_row dictionary with the calculated statistics and other details
    new_row = {
        'Event_Name': Event_Name,
        'Gridded_Data_Label': DSS_File.split('_')[0],
        'Event_Start_Date': dss_start_date,
        'Event_End_Date': dss_end_date,
        'Event_Duration': dss_duration,
        'Average_Precipitation': avg_precip_watershed,
        'Maximum_Precipitation': max_precip_watershed,
        'Minimum_Precipitation': min_precip_watershed
    }
    print(new_row)
    # Create a DataFrame from the new row
    new_row_df = pd.DataFrame([new_row], columns=columns)

    # Concatenate the existing DataFrame with the new row DataFrame
    statistics_df = pd.concat([statistics_df, new_row_df], ignore_index=True)

    # Save the DataFrame
    statistics_df.to_csv(csv_path, index=False)
    print("Statistics dataframe updated.")


    
    # Extract XY coordinates from the largest polygon's exterior
    coords = np.array(largest_polygon.geometry.exterior.coords)

    # Calculate min, and max for X and Y coordinates for plot extents

    # Calculate extents for the masked_precip
    y_size, x_size = masked_precip.shape
    x_min, y_max = out_transform * (0, 0)
    x_max, y_min = out_transform * (x_size, y_size)

    plot_extents_masked_precip = [x_min, x_max, y_min, y_max]

    # Plot masked points and watershed boundary
    fig, ax = plt.subplots(figsize=(5, 5))
    cax = ax.imshow(masked_precip, cmap='GnBu', extent=plot_extents_masked_precip)
    watershed_SHG.plot(ax=ax, facecolor='none', edgecolor='red')
    plt.colorbar(cax, ax=ax, label="Inches Precipitation")
    ax.set_title(f"Total Precipitation for {Event_Name}, {Gridded_Data_Label}, SHG")
    plt.show()
    fig.savefig(os.path.join(event_output_folder, "Report_Figure_TotalPrecip_Watershed_Boundary.png"))


    masked_total_precipitation_tiff_path = os.path.join(event_output_folder, "Masked_total_precipitation.tif")
    '''
    # Output masked_precip to a GeoTIFF file
    with rasterio.open(Total_precipitation_tiff_path) as src:
        profile = src.profile

    ...
    ...
    ...

    with rasterio.open(masked_total_precipitation_tiff_path, 'w', **profile) as dst:
        dst.write(out_image)
    print("Masked Total Precipitation GeoTIFF written to: ", masked_total_precipitation_tiff_path)
    '''

    # -------- Previous code block is to create masked_total_precipitation_tiff --------

    # Read GHNCD Daily Precip CSV from GHNCD Script output

    #Import Precip_CSV_Path as dataframe GHNCDailyPrecip_df
    GHNCDailyPrecip_df = pd.read_csv(Precip_CSV_Path, low_memory=False)
    #display(GHNCDailyPrecip_df)

    '''GHNCDailyPrecip_df sample:
        STATION	NAME	DATE	LATITUDE	LONGITUDE	PRCP	ELEVATION	PRCP_ATTRIBUTES	SNOW	SNOW_ATTRIBUTES	...	WT17	WT17_ATTRIBUTES	WT19	WT19_ATTRIBUTES	WT21	WT21_ATTRIBUTES	WT22	WT22_ATTRIBUTES	WV03	WV03_ATTRIBUTES
    0	US1LAAV0001	BUNKIE 0.3 WSW, LA US	2015-01-01	30.951738	-92.192111	3.0	24.1	,,N	NaN	NaN	...	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN
    1	US1LAAV0001	BUNKIE 0.3 WSW, LA US	2015-01-02	30.951738	-92.192111	23.0	24.1	,,N	NaN	NaN	...	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN
    '''

    # From Event Periods, create a start and end date for the daily data
    GHNCD_start_date = Event_Periods_dt[0][0].strftime("%Y-%m-%d")
    GHNCD_end_date = Event_Periods_dt[0][1].strftime("%Y-%m-%d")

    # Create an empty list to store data
    summed_data = []


    # Define transformers for coordinate conversion
    transformer = pyproj.Transformer.from_crs(4326, 5070, always_xy=True)

    # Loop through each unique station
    for station in GHNCDailyPrecip_df['STATION'].unique():
        # Filter GHNCDailyPrecip_df by station and date range
        GHNCDailyPrecip_df_station = GHNCDailyPrecip_df[GHNCDailyPrecip_df['STATION'] == station]
        latitude = GHNCDailyPrecip_df_station['LATITUDE'].iloc[0]
        longitude = GHNCDailyPrecip_df_station['LONGITUDE'].iloc[0]
        GHNCDailyPrecip_df_station_filtered = GHNCDailyPrecip_df_station[GHNCDailyPrecip_df_station['DATE'].between(GHNCD_start_date, GHNCD_end_date)]
        name = GHNCDailyPrecip_df_station['NAME'].iloc[0]

        # Get the sum of the PRCP column
        GHNCDailyPrecip_df_station_sum = GHNCDailyPrecip_df_station_filtered['PRCP'].sum()
        
        # Convert coordinates to SHG projection
        x, y = transformer.transform(longitude, latitude)
        
        # Append to the list
        summed_data.append({
            'STATION': station, 
            'NAME': name,
            'PRCP_SUM': GHNCDailyPrecip_df_station_sum,
            'LATITUDE': latitude,
            'LONGITUDE': longitude,
            'SHG X': x,
            'SHG Y': y
        })

    # Convert the list to a dataframe
    GHNCDailyPrecip_df_sum_event_period = pd.DataFrame(summed_data)

    # Convert from tenths of MM to inches
    GHNCDailyPrecip_df_sum_event_period['PRCP-IN'] = (GHNCDailyPrecip_df_sum_event_period['PRCP_SUM'] / 10) * 0.0393701

    # Display the new dataframe
    #display(GHNCDailyPrecip_df_sum_event_period)

    '''GHNCDailyPrecip_df_sum_event_period sample:
        STATION	PRCP_SUM	LATITUDE	LONGITUDE	SHG X	SHG Y	PRCP-IN
    0	US1LAAV0001	1446.0	30.951738	-92.192111	362681.496986	882366.707973	5.692916
    1	US1LABG0002	1170.0	30.471300	-93.159080	272210.873826	825771.822095	4.606302
    2	US1LACC0003	675.0	30.232246	-92.998641	288417.148472	799708.875420	2.657482
    '''

    # SHG is EPSG: 5070 (Standard Hydrologic Grid)
    # Using the "SHG X" and "SHG Y" columns from the dataframe GHNCDailyPrecip_df_sum_event_period, create a geodataframe with the points and PRCP-IN as the attribute



    # Convert the "SHG X" and "SHG Y" columns into a geometry column
    geometry = [Point(xy) for xy in zip(GHNCDailyPrecip_df_sum_event_period['SHG X'], GHNCDailyPrecip_df_sum_event_period['SHG Y'])]

    # Create a GeoDataFrame using the geometry column and PRCP-IN as the attribute
    geo_df = gpd.GeoDataFrame(GHNCDailyPrecip_df_sum_event_period, geometry=geometry, crs="EPSG:5070")

    # Display the first few rows of the GeoDataFrame
    geo_df.head()

    # -------- Previous code block is to create geo_df --------

    # Extract Gridded Precipitation from Total Precipitation TIFF file and Plot with GHNC Precipitation

    # Open the raster file and check its CRS
    with rasterio.open(Total_precipitation_tiff_path) as src:
        crs = src.crs

    print(f"The CRS of the raster file is: {crs}")


    # Load the raster data using rasterio
    with rasterio.open(Total_precipitation_tiff_path) as src:
        
        # Create a function to extract values from the raster at given x, y coordinates
        def extract_raster_value(x, y, raster):
            for val in raster.sample([(x, y)]):
                return val[0]

        # Apply the function to each station's coordinates to get the raster values
        geo_df['Grid_PRCP'] = geo_df.apply(lambda row: extract_raster_value(row['SHG X'], row['SHG Y'], src), axis=1)


    # Display geodataframe
    #display(geo_df)



    # Filter Geodataframe, excluding stations greater than 10 miles from the watershed boundary
    geo_df_filtered = geo_df[geo_df['geometry'].distance(largest_polygon.geometry) < GHNCD_Search_Buffer_Miles * 1609.34]

    # Filter Geodataframe, excluding stations with Grid_PRCP less than 0.01 inches
    geo_df_filtered = geo_df_filtered[geo_df_filtered['Grid_PRCP'] > 0.01]


    # Write geodataframe to shapefile
    geo_df_filtered.to_file(Precip_stations_shapefile_path)

    # Write Station, Name and PRCP-IN and Grid_PRCP to CSV
    geo_df_filtered[['STATION', 'NAME', 'SHG X', 'SHG Y', 'PRCP-IN', 'Grid_PRCP']].to_csv(Precip_stations_csv_path, index=False)

    

    # Existing extents of geo_df_filtered
    x_min_filtered = geo_df_filtered['SHG X'].min()
    x_max_filtered = geo_df_filtered['SHG X'].max()
    y_min_filtered = geo_df_filtered['SHG Y'].min()
    y_max_filtered = geo_df_filtered['SHG Y'].max()

    # Existing extents of watershed_SHG
    x_min_shg = watershed_shg_coords_before_rasterizing_extents[0] - GHNCD_Search_Buffer_Miles * 1609.34 / 1
    x_max_shg = watershed_shg_coords_before_rasterizing_extents[2] + GHNCD_Search_Buffer_Miles * 1609.34 / 1
    y_min_shg = watershed_shg_coords_before_rasterizing_extents[1] - GHNCD_Search_Buffer_Miles * 1609.34 / 1
    y_max_shg = watershed_shg_coords_before_rasterizing_extents[3] + GHNCD_Search_Buffer_Miles * 1609.34 / 1

    # Determine the greater extents for each boundary
    x_min = min(x_min_filtered, x_min_shg) - (increase_x_axis * 1609.34) + (shift_x_axis * 1609.34)
    x_max = max(x_max_filtered, x_max_shg) + (increase_x_axis * 1609.34) + (shift_x_axis * 1609.34)
    y_min = min(y_min_filtered, y_min_shg) - (plotextentsbuffer_miles) * 1609.34 + (shift_y_axis * 1609.34)
    y_max = max(y_max_filtered, y_max_shg) + (plotextentsbuffer_miles) * 1609.34 + (shift_y_axis * 1609.34)

    # Set the greater extents for the plot
    correct_plot_extents = [x_min, x_max, y_min, y_max]

    # Mask the total precipitation tiff file to the plot extents

    from shapely.geometry import box
    import geopandas as gpd
    import rasterio
    from rasterio.mask import mask

    # Create bounding box geometry from plot extents [minx, maxx, miny, maxy]
    bounding_box = box(correct_plot_extents[0], correct_plot_extents[2], correct_plot_extents[1], correct_plot_extents[3])

    # Create a GeoDataFrame with the bounding box
    bbox_gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=crs)  # Assuming crs is already defined in your code

    with rasterio.open(Total_precipitation_tiff_path) as src:
        # Convert bounding box to the same CRS as the raster
        bbox_gdf = bbox_gdf.to_crs(src.crs)
        
        # Mask the raster data with the bounding box
        out_image, out_transform = mask(src, bbox_gdf.geometry, crop=True)

    

    '''
    Example

    # Mask the total precipitation tiff file to the plot extents
    # This will ensure that the gridded data scales properly with the watershed shapefile

    # Create bounding box geometry from plot extents [minx, maxx, miny, maxy]
    bounding_box_report = box(plot_extents[0], plot_extents[2], plot_extents[1], plot_extents[3])

    # Create a GeoDataFrame with the bounding box
    bbox_gdf_report = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=total_precip_inches_tiff.rio.crs)

    # Clip the raster with the bounding box
    clipped_report_total_precip_inches_tiff = total_precip_inches_tiff.rio.clip(bbox_gdf.geometry)
    '''


    fig, ax = plt.subplots(figsize=(12, 9))

    # Plot the total precipitation TIFF file (masked)
    #cax = ax.imshow(out_image[0], cmap='GnBu', vmin=min_precip_watershed, vmax=max_precip_watershed, extent=correct_plot_extents)
    cax = ax.imshow(out_image[0], cmap='GnBu', vmin=min_precip_watershed, vmax=color_ramp_vmax, extent=correct_plot_extents)

    # Plot the watershed boundary
    watershed_SHG.plot(ax=ax, facecolor='none', edgecolor='red')
    # Plot the GeoDataFrame with station data
    geo_df_filtered.plot(ax=ax, color='black', markersize=7)
    # Add labels to each point (both original and AORC)
    for x, y, label, aorc, name in zip(geo_df_filtered['SHG X'], geo_df_filtered['SHG Y'], geo_df_filtered['PRCP-IN'], geo_df_filtered['Grid_PRCP'], geo_df_filtered['NAME']):
        ax.text(x, y, f'{label:.2f}  \n{name}  ', fontsize=6, ha='right', va='center')
        ax.text(x, y, f'   {aorc:.2f}\n  {Gridded_Data_Label}', fontsize=6, ha='left', va='center')
    # Set the extents based on the geo_df_filtered data
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    # Add a colorbar and title
    plt.colorbar(cax, ax=ax, label="Inches Precipitation")
    ax.set_title(Report_Figure_Title)
    # Show the combined plot
    plt.show()

    # Save plot to file Report_Figure_Path
    fig.savefig(Report_Figure_Path)
    display(geo_df_filtered)

    # Write Log File
    sys.stdout = sys.__stdout__
    log_contents = log_capture_string.getvalue()
    with open(log_file_path, 'w') as f:
        f.write(log_contents)
    log_contents = []

    # Close Rasterio Dataset
    rio_dataset.close()
    

# Save the DataFrame after the loop
statistics_df.to_csv(csv_path, index=False)
print("Statistics dataframe updated.")


In [ ]:
# Search all subfolders and child subfolders of output_files_folder for Precip_stations.csv


'''Example of Precip_stations.csv
STATION	NAME	SHG X	SHG Y	PRCP-IN	Grid_PRCP
US1LABG0002	RAGLEY 5.0 SE, LA US	272210.8738	825771.8221	0	5.4645705
USC00162641	DRY CREEK, LA US	273269.0971	855829.9724	5.30315247	6.3082705
USC00165266	LEESVILLE, LA US	262324.2722	900025.6272	9.17717031	9.112999
USC00166938	OBERLIN FIRE TOWER, LA US	308605.782	841640.1602	5.29134144	4.7503963
'''



# Compile Event CSV Outputs for GHNCD vs Gridded Comparisons
import os
import pandas as pd

def search_csv_files(folder):
    csv_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith("Precip_stations.csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

output_files_folder = os.path.join(os.path.dirname(Watershed_Shapefile), "Output_Files")
def main():
    csv_files = search_csv_files(output_files_folder)
    
    all_dataframes = []
    
    for csv_file in csv_files:
        # Read the CSV files into dataframes
        df = pd.read_csv(csv_file)
        
        # Using the parent folder name as the Event Name, create a new column in the dataframe
        parent_folder = os.path.basename(os.path.dirname(csv_file))
        df['Event Name'] = parent_folder
        
        all_dataframes.append(df)
    
    # Concatenate the dataframes into one dataframe
    final_dataframe = pd.concat(all_dataframes, ignore_index=True)
    
    # Calculate PRCP-IN minus Grid_PRCP and add a new column to the dataframe named "GHNCD - GRIDDED"
    final_dataframe["GHNCD - GRIDDED"] = final_dataframe["PRCP-IN"] - final_dataframe["Grid_PRCP"]
    
    # Save the dataframe to a CSV file in the output_files_folder named "GHNCD_vs_Gridded.csv"
    final_dataframe.to_csv(os.path.join(output_files_folder, "GHNCD_vs_Gridded.csv"), index=False)

if __name__ == "__main__":
    main()
